In [ ]:
from pyvirtualdisplay import Display

virtual_display = Display(visible=False, size=(1400, 900))
virtual_display.start()

In [9]:
import gym

from huggingface_sb3 import load_from_hub, package_to_hub, push_to_hub
from huggingface_hub import (
    notebook_login,
)

from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env

In [2]:
import gym

env = gym.make("LunarLander-v2")

observation = env.reset()

for _ in range(20):
    action = env.action_space.sample()
    print(action)

    observation, reward, done, info = env.step(action)

    if done:
        print('done.')
        observation = env.reset()

0
3
2
2
2
3
3
3
3
1
1
0
0
2
2
3
1
3
1
3


In [7]:
env.reset()

print('Observation space shape:', env.observation_space.shape)
print('Sample observation: ', env.observation_space.sample())

print('Action space shape:', env.action_space.n)
print('Sample action: ', env.action_space.sample())

Observation space shape: (8,)
Sample observation:  [-0.8589128   1.241753    1.2944345  -0.01915429  0.07398096 -0.03286204
  0.39541364 -0.06663246]
Action space shape: 4
Sample action:  0


In [10]:
model = PPO(
    policy='MlpPolicy',
    env=env,
    n_steps=2048,
    batch_size=64,
    n_epochs=4,
    gamma=0.999,
    gae_lambda=0.98,
    ent_coef=0.01,
    verbose=1,
)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [15]:
model.learn(total_timesteps=10000)

model_name = "ppo-lunarlander-v2"
model.save(model_name)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 102      |
|    ep_rew_mean     | -91.4    |
| time/              |          |
|    fps             | 1303     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 108         |
|    ep_rew_mean          | -86.6       |
| time/                   |             |
|    fps                  | 1059        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.011711956 |
|    clip_fraction        | 0.071       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.3        |
|    explained_variance   | -0.00224    |
|    learning_rate        | 0.

In [ ]:
eval_env = gym.make("LunarLander-v2")

mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward}")

In [13]:
notebook_login()

In [ ]:
import gym
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env

from huggingface_sb3 import package_to_hub

repo_id = "akghxhs55/ppo-lunarlander-v2"

env_id = "LunarLander-v2"

eval_env = DummyVecEnv([lambda: gym.make(env_id)])


model_architecture = "PPO"

commit_message = "initial commit"

package_to_hub(model=model,
               model_name=model_name,
               model_architecture=model_architecture,
               env_id=env_id,
               eval_env=eval_env,
               repo_id=repo_id,
               commit_message=commit_message,
)